In [ ]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import DictionaryLearning

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../test/dataset/meas_symm_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_2.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_3.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_4.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_5.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_symm_varspeed_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_2.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_reflector_1.npz', allow_pickle=False)
# measurement = np.load('../test/dataset/meas_asymm_reflector_2.npz', allow_pickle=False)

header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]
# First, split into train (60%) and temp (40%) sets
trainCIR, tempCIR = train_test_split(data_cir, test_size=0.4, random_state=42)

# Now, split tempCIR into test (20%) and evaluation (20%) sets
testCIR, evalCIR = train_test_split(tempCIR, test_size=0.5, random_state=42)

print(f"Train Data: {trainCIR.shape}")
print(f"Test Data: {testCIR.shape}")
print(f"Evaluation Data: {evalCIR.shape}")

num = 5
n_comp = num
no_nonzero_coefs = num

Train Data: (600, 15, 251, 2)
Test Data: (200, 15, 251, 2)
Evaluation Data: (200, 15, 251, 2)


In [ ]:
# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

def apply_pca(data, n_components):
    # data: (samples, 251, 2)
    reshaped_data = data.reshape(data.shape[0], -1)
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(reshaped_data)
    
    pca = PCA(n_components=n_components)
    data_pca = pca.fit_transform(data_scaled)  # (samples, n_components)
    
    return data_pca, scaler, pca

train_cirs_pca, scaler, pca = apply_pca(train_data_combined, n_components=n_comp)

# Create labels for Alice and Eve for training
alice_train_labels = np.zeros(train_alice_magnitude.shape[0])  # Label '0' for Alice.
eve_train_labels = np.ones(train_eve_magnitude.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((alice_train_labels, eve_train_labels))

# Combine data and labels for training
train_atoms = train_cirs_pca
# Step 4: Form the Dictionary D from Training Data
D = train_atoms.T

reshaped_data: (1200, 251)
data_scaled: (1200, 251)
train_cirs_pca: (1200, 5)
(1200, 5)


In [144]:
# # Step 3: Extract Features for Test Data
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]
# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_cirs = np.vstack((test_alice_magnitude, test_eve_magnitude))

# Scale
test_cirs_scaled = scaler.transform(test_cirs)
print(f'test_cirs_scaled: {test_cirs_scaled.shape}')
# PCA
test_cirs_pca = pca.transform(test_cirs_scaled)
print(f'test_cirs_pca: {test_cirs_pca.shape}')

# Create labels for Alice and Eve for testing
alice_test_labels = np.zeros(test_alice_magnitude.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(test_eve_magnitude.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((alice_test_labels, eve_test_labels))

# Combine data and labels for testing
test_atoms = test_cirs_pca

test_cirs_scaled: (400, 251)
test_cirs_pca: (400, 5)


In [ ]:
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=no_nonzero_coefs):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, coefficients, class_indices, D):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    coefficients = find_sparse_coefficients(tSample, D)
    
    residuals = []

    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, coefficients, class_indices, D)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class

In [146]:
# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for testSample in test_atoms:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)


predictions = np.array(predictions)
# print(predictions.shape)


accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

Classification Accuracy: 51.00%


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

In [ ]:
# -------------------------------------------------------- Testing ------------------------------------------------
# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

Classification Accuracy: 51.00%
tp: 158
tn: 46
fp: 154
fn: 42
MDR: 0.77
FAR: 0.21
AR: 0.51


In [ ]:
# -------------------------------------------------------- Evaluation  ------------------------------------------------
# Extract CIRs
eval_alice_CIRs = evalCIR[:, alice_channel, :, :]
eval_eve_CIRs = evalCIR[:, eve_channel, :, :]

# Compute magnitudes
eval_alice_magnitude = np.abs(eval_alice_CIRs[..., 0] + 1j * eval_alice_CIRs[..., 1])
eval_eve_magnitude = np.abs(eval_eve_CIRs[..., 0] + 1j * eval_eve_CIRs[..., 1])

# Stack evaluation features
eval_atoms = np.vstack((eval_alice_magnitude, eval_eve_magnitude))

# Scale
eval_scaled = scaler.transform(test_cirs)
print(f'test_cirs_scaled: {eval_scaled.shape}')
# PCA
eval_cirs = pca.transform(eval_scaled)
print(f'test_cirs_pca: {eval_cirs.shape}')

# Labels for evaluation data
eval_alice_labels = np.zeros(eval_alice_magnitude.shape[0])  # Label '0' for Alice
eval_eve_labels = np.ones(eval_eve_magnitude.shape[0])       # Label '1' for Eve
eval_labels = np.hstack((eval_alice_labels, eval_eve_labels))

# Step 8: Classifying Evaluation Data
eval_predictions = []

for evalSample in eval_cirs:
    predicted_class = classify_signal(evalSample, D, train_labels)
    eval_predictions.append(predicted_class)

eval_predictions = np.array(eval_predictions)

# Calculate accuracy
eval_accuracy = accuracy_score(eval_labels, eval_predictions)
print(f"Classification Accuracy: {eval_accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(eval_labels, eval_predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")



/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

test_cirs_scaled: (400, 251)
test_cirs_pca: (400, 5)


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

Classification Accuracy: 51.00%
tp: 158
tn: 46
fp: 154
fn: 42
MDR: 0.77
FAR: 0.21
AR: 0.51


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend